## Homework

In this homework, we'll deploy Credit Card prediction model from the homework 5.
We already have a docker image for this model - we'll use it for 
deploying the model to Kubernetes.

Go to the `Week 10/service` folder and 
execute the following:


```bash
docker build -t zoomcamp-model:v001 .
```

## Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:v001
```

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```
{'get_card': True, 'get_card_probability': <value>}
```

Here `<value>` is the probability of getting a credit card. You need to choose the right one.

Here `<value>` is the probability of getting a credit card. You need to choose the right one.

* 0.289
* 0.502
* 0.769
* 0.972

In [13]:
#Result
!python -m app.q6_test

{'get_card': True, 'get_card_probability': 0.7692649226628628}


`Answer`: 0.769

## Question 2

What's the version of `kind` that you have? 

Use `kind --version` to find out.

In [3]:
!kind --version

kind version 0.17.0


## Creating a cluster

Now let's create a cluster with `kind`:

```bash
kind create cluster
```

And check with `kubectl` that it was successfully created:

```bash
kubectl cluster-info

## Question 3

What's the smallest deployable computing unit that we can create and manage 
in Kubernetes (`kind` in our case)?

* Node
* Pod
* Deployment
* Service

`Answer`:Pods are the smallest deployable units of computing that you can create and manage in Kubernetes. 

https://kubernetes.io/docs/concepts/workloads/pods/

## Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the `Type` of the service that is already running there?

* ClusterIP
* NodePort
* LoadBalancer
* ExternalName

In [7]:
!kubectl get service

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   13h


`Answer`: ClusterIP

### Create kubernetes cluster
- kubectl apply -f deployment.yaml
- kind load docker-image zoomcamp-model:v001
- kubectl get pod

In [10]:
!kubectl get pod

NAME                           READY   STATUS    RESTARTS   AGE
credit-card-854fcccb98-97szn   1/1     Running   0          3m2s
service-7f996b9c65-58bmn       1/1     Running   0          13s


## Question 5

To be able to use the docker image we previously created (`zoomcamp-model:v001`),
we need to register it with `kind`.

What's the command we need to run for that?

* `kind create cluster`
* `kind build node-image`
* `kind load docker-image`
* `kubectl apply`

`Answer`: kind load docker-image

## Question 6

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: credit-card
spec:
  selector:
    matchLabels:
      app: credit-card
  replicas: 1
  template:
    metadata:
      labels:
        app: credit-card
    spec:
      containers:
      - name: credit-card
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`?

Apply this deployment using the appropriate command and get a list of running Pods. 
You can see one running Pod.

`Answer`: 9696 according to the Dockerfile

## Question 7

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?

Apply this config file

`Answer`: credit-card

# Activate Service

- kubectl apply -f service.yaml
- kubectl port-forward service/credit-score 9696:80

## Autoscaling

Now we're going to use a [HorizontalPodAutoscaler](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale-walkthrough/) 
(HPA for short) that automatically updates a workload resource (such as our deployment), 
with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

```bash
kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3
```

You can check the current status of the new HPA by running:

```bash
kubectl get hpa
```

The output should be similar to the next:

```bash
NAME              REFERENCE                TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
credit-card-hpa   Deployment/credit-card   1%/20%    1         3         1          27s
```

`TARGET` column shows the average CPU consumption across all the Pods controlled by the corresponding deployment.
Current CPU consumption is about 0% as there are no clients sending requests to the server.
> 
>Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release 
> from the `components.yaml` manifest:
> ```bash
> kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
>```

## Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this 
run many_request.py script


## Question 8 (optional)

Run `kubectl get hpa credit-card-hpa --watch` command to monitor how the autoscaler performs. 
Within a minute or so, you should see the higher CPU load; and then - more replicas. 
What was the maximum amount of the replicas during this test?


* 1
* 2
* 3
* 4

In [ ]:
!python -m app.many_request